<a href="https://colab.research.google.com/github/MittalNeha/AI-ML_classProjects/blob/master/Github_Diffusion_Training_CLIP_Conditional.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Train a Text Conditional Image Generation using Diffusion Models:

- To see more details on the code and model training visit the [Github Repo](https://github.com/apapiu/guided-diffusion-keras)

Make sure to go to Runtime and set your Hardware -> Change Runtime Type and have the accelerator set to GPU!

Note that this notebook does **not** read or write anything to your machine nor does it need any access to google drive or outside sources. All the downloads happen within this notebook and once you close it, it will all go gone poof!

- Author: Alexandru Papiu
- Email: alexpapiu@gmail.com


In [1]:
# @title Licensed under the MIT License

# Copyright (c) 2022 Alexandru Papiu

# Permission is hereby granted, free of charge, to any person obtaining a copy
# of this software and associated documentation files (the "Software"), to deal
# in the Software without restriction, including without limitation the rights
# to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
# copies of the Software, and to permit persons to whom the Software is
# furnished to do so, subject to the following conditions:

# The above copyright notice and this permission notice shall be included in
# all copies or substantial portions of the Software.

# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
# AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
# OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN
# THE SOFTWARE.

In [ ]:
#clone the repo so you can mess around with it:
!git clone https://github.com/apapiu/guided-diffusion-keras.git
! pip install git+https://github.com/openai/CLIP.git

#this is where the data is:
!git lfs install
!git clone https://huggingface.co/apapiu/diffusion_model_aesthetic_keras

import os
import sys
import numpy as np
import pandas as pd

from tensorflow.keras.datasets import mnist, fashion_mnist, cifar10, cifar100
from keras.utils.vis_utils import plot_model

from tensorflow import keras
from matplotlib import pyplot as plt

####local imports:
sys.path.append("/content/guided-diffusion-keras/guided_diffusion")
from denoiser import get_network
from utils import batch_generator, plot_images, get_data, preprocess
from diffuser import Diffuser

Cloning into 'guided-diffusion-keras'...
remote: Enumerating objects: 121, done.
remote: Counting objects: 100% (121/121), done.
remote: Compressing objects: 100% (93/93), done.
Receiving objects: 100% (121/121), 40.59 KiB | 10.15 MiB/s, done.
remote: Total 121 (delta 73), reused 61 (delta 27), pack-reused 0
Resolving deltas: 100% (73/73), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-ore4zycp
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-ore4zycp
  Resolved https://github.com/openai/CLIP.git to commit a9b1bf5920416aaeaec965c25dd9e8f98c864f16
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 1.8 MB/s eta 0:00:00
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369370 sha256=6c19b0c56046faf2f5a5e6943b04e3daeb83510e362ff3baa2eba1437e81

In [ ]:
#########
# CONFIG:
#########

train_model=True

image_size = 64
num_channels = 3
epochs = 50
class_guidance = 4

# architecture
channels = 96
channel_multiplier = [1, 2, 3, 4]
block_depth = 2
emb_size = 512  # CLIP/label embedding
num_classes = 12  # only used if precomputed_embedding=False
attention_levels = [0, 0, 1, 0]

embedding_dims = 32
embedding_max_frequency = 1000.0

precomputed_embedding = True
save_in_drive = False
widths = [c * channels for c in channel_multiplier]

###train process config:
batch_size = 64
num_imgs = 36 #num imgs to test on - should be a square - 25, 64, 100 etc.
row = int(np.sqrt(num_imgs))

validation_num = 100 #don't train on first validation_num images so we can use as visual validation
train_size = 40000 #only train on a subset of train data - do this for faster results..

learning_rate = 0.0003

MODEL_NAME = "model_test_aesthetic"
from_scratch = True #if False will load model from model path and continue training
file_name = "from_huggingface"

data_dir = '/content/diffusion_model_aesthetic_keras'
captions_path = os.path.join(data_dir, "captions.csv")
imgs_path = os.path.join(data_dir, "imgs.npy")
embedding_path = os.path.join(data_dir, "embeddings.npy")

if save_in_drive:
    from google.colab import drive
    drive.mount('/content/drive')
    drive_path = '/content/drive/MyDrive/'
    home_dir = os.path.join(drive_path, MODEL_NAME)
else:
    home_dir = MODEL_NAME

if not os.path.exists(home_dir):
    os.mkdir(home_dir)

model_path = os.path.join(home_dir, MODEL_NAME + ".h5")

In [ ]:
##################################
###########Loading Data And Model:
##################################

#load the data from a numpy file file:
captions = pd.read_csv(captions_path)
train_data, train_label_embeddings = np.load(imgs_path), np.load(embedding_path)
#train_data, train_label_embeddings = get_data(npz_file_name=file_name, prop=0.6, captions=False)
print(train_data.shape)

labels = train_label_embeddings[:num_imgs]

np.random.seed(100)
rand_image = np.random.normal(0, 1, (num_imgs, image_size, image_size, num_channels))

if from_scratch:
    autoencoder = get_network(image_size,
                              widths,
                              block_depth,
                              num_classes=num_classes,
                              attention_levels=attention_levels,
                              emb_size=emb_size,
                              num_channels=num_channels,
                              precomputed_embedding=precomputed_embedding)

    autoencoder.compile(optimizer="adam", loss="mae")
else:
    autoencoder = keras.models.load_model(model_path)


##################
#Some data checks:
##################

print("Number of pamaters is {0}".format(autoencoder.count_params()))
pd.Series(train_data[:1000].ravel()).hist(bins=80)
plt.show()
print("Original Validation Images:")
plot_images(preprocess(train_data[:num_imgs]), nrows=int(np.sqrt(num_imgs)))
plot_model(autoencoder, to_file=os.path.join(home_dir, 'model_plot.png'),
           show_shapes=True, show_layer_names=True)
print("Validation Captions:")
print(captions[:num_imgs]["0"].values)
print("Generating Images below:")


In [ ]:
#############################
#!create generator and train:
#############################

diffuser = Diffuser(autoencoder,
                    class_guidance=class_guidance,
                    diffusion_steps=35)

if train_model:
    train_generator = batch_generator(autoencoder,
                                    model_path,
                                    train_data[validation_num:validation_num+train_size],
                                    train_label_embeddings[validation_num:validation_num+train_size],
                                    epochs,
                                    batch_size,
                                    rand_image,
                                    labels,
                                    home_dir,
                                    diffuser)

    autoencoder.optimizer.learning_rate.assign(learning_rate)

    eval_nums = autoencoder.fit(
        x=train_generator,
        epochs=epochs,
    )